In [1]:
# CNN_v4.py - 90%+ EMNIST Character Classifier with Progressive Fine-Tuning
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pickle

from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:

from sklearn.utils import class_weight

# ========== Load EMNIST Dataset ==========
(ds_train, ds_test), ds_info = tfds.load(
    'emnist/byclass',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True
)

NUM_CLASSES = ds_info.features['label'].num_classes
IMG_SIZE = 224
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE
label_map = ds_info.features['label'].int2str

# ========== Preprocessing ==========
def preprocess(image, label):
    image = tf.image.rot90(image, k=1)
    image = tf.image.flip_left_right(image)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.grayscale_to_rgb(image)
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

# ========== Augmentation ==========
augment = tf.keras.Sequential([
    layers.RandomRotation(0.15),
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomZoom(0.1, 0.1),
    layers.RandomContrast(0.1)
])

def augment_fn(image, label):
    return augment(image), label

# ========== Datasets ==========
train_ds = ds_train.map(preprocess).map(augment_fn).shuffle(2048).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_ds = ds_test.map(preprocess).batch(BATCH_SIZE).prefetch(AUTOTUNE)

# ========== Compute Class Weights ==========
labels_np = np.concatenate([
    np.argmax(y, axis=-1)
    for _, y in tfds.as_numpy(ds_train.map(preprocess).batch(1024))
])

class_weights_arr = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.arange(NUM_CLASSES),
    y=labels_np
)
class_weights = dict(enumerate(class_weights_arr))

# ========== Base Models ==========
input_layer = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_effnet = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_layer)
base_resnet = ResNet50(include_top=False, weights='imagenet', input_tensor=input_layer)

# Freeze all layers initially
base_effnet.trainable = False
base_resnet.trainable = False

# EfficientNet branch
x1 = base_effnet.output
x1 = layers.GlobalAveragePooling2D()(x1)

# ResNet branch
x2 = base_resnet.output
x2 = layers.GlobalAveragePooling2D()(x2)

# Merge branches
combined = layers.Concatenate()([x1, x2])
combined = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(combined)
combined = layers.BatchNormalization()(combined)
combined = layers.Dropout(0.5)(combined)
output = layers.Dense(NUM_CLASSES, activation='softmax')(combined)

model = models.Model(inputs=input_layer, outputs=output)

# ========== Compile (Initial Training) ==========
model.compile(
    optimizer=Adam(learning_rate=0.0003),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# ========== Callbacks ==========
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=2, factor=0.3, min_lr=1e-6),
    ModelCheckpoint("EMNIST_V3_best_model.h5", save_best_only=True)
]

# ========== Initial Training ==========
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=20,
    callbacks=callbacks,
    class_weight=class_weights
)

# ========== Save Initial History ==========
with open("emnist_V3_byclass_stage1.pkl", "wb") as f:
    pickle.dump(history.history, f)

# ========== Fine-Tuning ==========
# Unfreeze last N layers of both models
N_EFFNET = 20
N_RESNET = 30

for layer in base_effnet.layers[-N_EFFNET:]:
    layer.trainable = True

for layer in base_resnet.layers[-N_RESNET:]:
    layer.trainable = True

# Re-compile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower LR for fine-tuning
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# ========== Fine-Tune Training ==========
fine_tune_history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10,
    callbacks=callbacks,
    class_weight=class_weights
)

# ========== Save Fine-Tune History ==========
with open("emnist_V3_byclass_history.pkl", "wb") as f:
    pickle.dump(fine_tune_history.history, f)

# ========== Evaluation ==========
preds = model.predict(test_ds)
y_true = np.argmax(np.concatenate([y for _, y in test_ds], axis=0), axis=1)
y_pred = np.argmax(preds, axis=1)

print("Classification Report:\n")
print(classification_report(y_true, y_pred))

# ========== Confusion Matrix ==========
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(20, 20))
sns.heatmap(cm, cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# ========== Visualize Predictions ==========
for images, labels in test_ds.take(1):
    pred_probs = model.predict(images)
    for i in range(25):
        idx = random.randint(0, len(images)-1)
        img = images[idx].numpy()
        true_lbl = label_map(np.argmax(labels[idx].numpy()))
        pred_lbl = label_map(np.argmax(pred_probs[idx]))
        plt.subplot(5, 5, i+1)
        plt.imshow(img)
        plt.title(f"T:{true_lbl}\nP:{pred_lbl}")
        plt.axis('off')
plt.tight_layout()
plt.show()


Epoch 1/20
   23/10906 ━━━━━━━━━━━━━━━━━━━━ 9:18:52 3s/step - accuracy: 0.0265 - loss: 4.8177